In [40]:
%%bash
pip3 install psycopg2-binary
pip3 install networkx
pip3 install colorama
pip3 install termcolor

In [45]:
import subprocess
import psycopg2
from pathlib import Path
import json
import ctdPython
from ctdPython.hypergraph import HyperGraph
from ctdPython.ctdcheck import CTDCheck

In [47]:
REWRITE_JAR = 'rewrite-assembly-0.1.0-SNAPSHOT.jar'

class Rewriting:
    def __init__(self, original, rewritten, features, time, drop_statements):
        self.original = original
        self.rewritten = rewritten
        self.features = features
        self.time = time
        self.drop_statements = drop_statements

class QueryRewriter:
    def __init__(self, host, database, user, password, port=5432):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port
        self.jdbcString = f'jdbc:postgresql://{self.host}:{self.port}/{self.database}'

    def rewrite(self, query):
        rewriteProcess = subprocess.run(['java', '-jar', REWRITE_JAR, query,
                                         self.jdbcString, self.database, self.user, self.password])
        print(rewriteProcess.returncode)
        if (rewriteProcess.returncode != 0):
            print(f'Error calling {rewriteProcess}')
            return None

        output = json.loads(Path('output/output.json').read_text())
        drop_output = json.loads(Path('output/drop.json').read_text())

        result = Rewriting(query, output['rewritten_query'], output['features'], output['time'],
                           drop_output['rewritten_query'])

        hg = HyperGraph.fromHyperbench('output/hypergraph.txt')
        print('hg: ' + str(hg))
        ctd = CTDCheck(hg)
        print('ctd: ' + str(ctd))
        return result

query = """
SELECT MIN(p.Id) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.AnswerCount>=0 AND p.FavoriteCount>=0 AND pl.LinkTypeId=1 AND ph.PostHistoryTypeId=2 AND v.CreationDate>=CAST('2010-07-20 00:00:00' AS TIMESTAMP) AND u.Reputation>=1 AND u.DownVotes>=0 AND u.DownVotes<=0 AND u.UpVotes<=439 AND u.CreationDate<=CAST('2014-08-07 11:18:45' AS TIMESTAMP)
"""

rewriter = QueryRewriter('postgres', 'stats', 'stats', 'stats')

result = rewriter.rewrite(query)
print(result)

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


org.apache.calcite.jdbc.CalciteSchema$SchemaPlusImpl@771d1ffbLogicalAggregate(group=[{}], EXPR$0=[MIN($0)])
  LogicalProject(id=[$0])
    LogicalJoin(condition=[=($5, $29)], joinType=[inner])
      LogicalJoin(condition=[=($5, $27)], joinType=[inner])
        LogicalJoin(condition=[=($5, $24)], joinType=[inner])
          LogicalJoin(condition=[=($5, $19)], joinType=[inner])
            LogicalJoin(condition=[=($0, $13)], joinType=[inner])
              LogicalFilter(condition=[AND(>=($6, 0), >=($8, 0))])
                JdbcTableScan(table=[[stats, posts]])
              LogicalFilter(condition=[=(CAST($4):INTEGER, 1)])
                JdbcTableScan(table=[[stats, postlinks]])
            LogicalFilter(condition=[=(CAST($1):INTEGER, 2)])
              JdbcTableScan(table=[[stats, posthistory]])
          LogicalFilter(condition=[>=($3, 2010-07-20 00:00:00)])
            JdbcTableScan(table=[[stats, votes]])
        JdbcTableScan(table=[[stats, badges]])
      LogicalFilter(condition=[

In [ ]:
def run_rewritten(query):
    conn = psycopg2.connect(
        host="postgres",
        database=database,
        user=database,
        password=database
    )